In [ ]:
#started 10-14-2025

In [ ]:
# citation: United States Geological Survey (2021). United States Geological Survey 3D Elevation
#  Program 1 arc-second Digital Elevation Model. Distributed by OpenTopography. https://doi.org/10.5069/G9HX19WN. Accessed 2025-10-14

In [1]:
import pandas as pd
import rasterio
from rasterio import sample
import numpy as np
from tqdm import tqdm



In [2]:
fires = pd.read_csv("../attempt 4/fires_with_pop.csv")

In [3]:
fires.describe()

,OBJECTID,FIRE_YEAR,DISCOVERY_DATE,FIRE_SIZE,LATITUDE,LONGITUDE,OBJECTID.1,temp_max_F,humidity_pct,precip_in,windspeed_mph,ndvi,pop_density
count,2.926000e+03,2926.000000,2.926000e+03,2926.000000,2926.000000,2926.000000,2.926000e+03,2926.000000,2926.000000,2926.000000,2926.000000,2925.000000,2926.000000
mean,1.119678e+06,2009.356801,2.455157e+06,698.459740,33.847226,-117.369907,1.119678e+06,83.949132,68.571770,0.003938,8.036604,3166.863248,261.331656
std,6.234096e+05,3.011743,1.093919e+03,8030.910191,0.616454,0.966675,6.234096e+05,11.231815,19.486523,0.020734,2.537002,1137.025054,652.147650
min,1.100000e+02,2005.000000,2.453406e+06,2.000000,32.552700,-120.574722,1.100000e+02,43.340000,9.000000,0.000000,2.548167,-3000.000000,0.000000
25%,3.691762e+05,2007.000000,2.454122e+06,3.000000,33.521111,-117.776414,3.691762e+05,76.820000,54.000000,0.000000,6.401492,2331.000000,0.454851
50%,1.181302e+06,2009.000000,2.455136e+06,6.300000,33.896111,-117.182678,1.181302e+06,85.100000,71.000000,0.000000,7.551274,3020.000000,13.918338
75%,1.626262e+06,2012.000000,2.456067e+06,32.000000,34.311111,-116.862188,1.626262e+06,91.940000,85.000000,0.000000,9.058421,3923.000000,133.882072
max,1.880442e+06,2015.000000,2.457382e+06,240207.000000,35.000000,-114.203000,1.880442e+06,118.400000,100.000000,0.551181,28.589186,6407.000000,8307.394531


In [23]:
raster_path = "output_USGS30m.tif"
raster = rasterio.open(raster_path)

In [24]:
print("Raster width, height:", raster.width, raster.height)
print("Raster bounds:", raster.bounds)
print("Raster CRS:", raster.crs)
print("Number of bands:", raster.count)

Raster width, height: 27440 11244
Raster bounds: BoundingBox(left=-121.23083286318197, bottom=32.278610697325874, right=-113.60861057998186, top=35.40194405564592)
Raster CRS: EPSG:4269
Number of bands: 1


In [ ]:
raster_array = raster.read(1, masked=True)

slope_values = []

for lon, lat in tqdm(zip(fires['LONGITUDE'], fires['LATITUDE']), 
                     total=len(fires), desc="Extracting slope values"):
    try:
        # Ensure longitude is negative for western hemisphere
        lon = -abs(lon)

        # Get raster indexes (row, col)
        row, col = raster.index(lon, lat)

        # Extract slope value from raster
        slope_values.append(raster_array[row, col])
    except IndexError:
        # If the coordinate falls outside the raster bounds
        slope_values.append(np.nan)

# Convert masked pixels (NoData) to nan
slope_values = np.array([
    val if not np.ma.is_masked(val) else np.nan 
    for val in slope_values
])

# Assign to the df
fires["slope"] = slope_values

Extracting slope values: 100%|██████████| 2926/2926 [00:00<00:00, 22041.74it/s]


In [26]:
fires.head()

,OBJECTID,FIRE_YEAR,DISCOVERY_DATE,FIRE_SIZE,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,OBJECTID.1,temp_max_F,humidity_pct,precip_in,windspeed_mph,time,ndvi,pop_density,slope
0,110,2005,2453540.5,10.0,Equipment Use,33.718889,-117.433611,110,73.04,89,0.062992,6.028589,1970-01-01 00:00:00.002453540,5016.0,1631.960938,543.523010
1,155,2005,2453411.5,3.0,Debris Burning,34.748333,-119.410278,155,58.46,79,0.000000,4.536979,1970-01-01 00:00:00.002453411,3357.0,0.696928,1002.209473
2,178,2005,2453544.5,4.2,Equipment Use,34.466667,-119.828333,178,77.54,76,0.000000,15.972654,1970-01-01 00:00:00.002453544,4356.0,24.228647,102.410408
3,1053,2005,2453559.5,3.0,Miscellaneous,34.479444,-118.768611,1053,82.76,86,0.000000,6.028589,1970-01-01 00:00:00.002453559,3124.0,0.044615,335.076935
4,1282,2005,2453582.5,2.0,Lightning,33.110833,-116.847222,1282,87.62,88,0.000000,8.763207,1970-01-01 00:00:00.002453582,4180.0,0.154729,337.525696


In [27]:
fires.describe()

,OBJECTID,FIRE_YEAR,DISCOVERY_DATE,FIRE_SIZE,LATITUDE,LONGITUDE,OBJECTID.1,temp_max_F,humidity_pct,precip_in,windspeed_mph,ndvi,pop_density,slope
count,2.926000e+03,2926.000000,2.926000e+03,2926.000000,2926.000000,2926.000000,2.926000e+03,2926.000000,2926.000000,2926.000000,2926.000000,2925.000000,2926.000000,2926.000000
mean,1.119678e+06,2009.356801,2.455157e+06,698.459740,33.847226,-117.369907,1.119678e+06,83.949132,68.571770,0.003938,8.036604,3166.863248,261.331656,624.748535
std,6.234096e+05,3.011743,1.093919e+03,8030.910191,0.616454,0.966675,6.234096e+05,11.231815,19.486523,0.020734,2.537002,1137.025054,652.147650,416.513489
min,1.100000e+02,2005.000000,2.453406e+06,2.000000,32.552700,-120.574722,1.100000e+02,43.340000,9.000000,0.000000,2.548167,-3000.000000,0.000000,-70.337067
25%,3.691762e+05,2007.000000,2.454122e+06,3.000000,33.521111,-117.776414,3.691762e+05,76.820000,54.000000,0.000000,6.401492,2331.000000,0.454851,340.791969
50%,1.181302e+06,2009.000000,2.455136e+06,6.300000,33.896111,-117.182678,1.181302e+06,85.100000,71.000000,0.000000,7.551274,3020.000000,13.918338,543.042908
75%,1.626262e+06,2012.000000,2.456067e+06,32.000000,34.311111,-116.862188,1.626262e+06,91.940000,85.000000,0.000000,9.058421,3923.000000,133.882072,860.807877
max,1.880442e+06,2015.000000,2.457382e+06,240207.000000,35.000000,-114.203000,1.880442e+06,118.400000,100.000000,0.551181,28.589186,6407.000000,8307.394531,3008.082764


In [ ]:
# it looks like slope values are percent slope, so like rise/run * 100. this means a slope value of 567 means 5.67% slope

In [28]:
fires.columns

Index(['OBJECTID', 'FIRE_YEAR', 'DISCOVERY_DATE', 'FIRE_SIZE',
       'STAT_CAUSE_DESCR', 'LATITUDE', 'LONGITUDE', 'OBJECTID.1', 'temp_max_F',
       'humidity_pct', 'precip_in', 'windspeed_mph', 'time', 'ndvi',
       'pop_density', 'slope'],
      dtype='object')

In [29]:
fires.to_csv("fires_with_slope.csv", index=False)